In [67]:
import pandas as pd
import requests
import json
from pprint import pprint
from config import yelp_key, g_key
import numpy as np
import gmaps
from scipy.stats import linregress
import matplotlib.pyplot as plt

In [32]:
#business search

headers = {'Authorization': 'bearer %s' % yelp_key}

#url & parameters--https://www.yelp.com/developers/documentation/v3/business_search

params = {'location': 'monterrey',
          'term': 'hotel',
         "limit": 50}

url = "https://api.yelp.com/v3/businesses/search"

#------------------

resp = requests.get(url=url, params=params, headers=headers).json()

In [35]:
#info recollection function

def yelp_business_call(response):

    business_list = []

    for business in response["businesses"]:

        try:

            name = business["name"]
            price = business["price"]
            rating = business["rating"]
            review_count = business["review_count"]
            city = business["location"]["city"]
            country = business["location"]["country"]
            latitude = business["coordinates"]["latitude"]
            longitude = business["coordinates"]["longitude"]
            categories = [category["alias"] for category in business["categories"]]

            business_dict = {"Name": name,
                            "Price": price,
                            "Rating": rating,
                            "Review Count": review_count,
                             "City": city,
                             "Country": country,
                             "Latitude": latitude,
                             "Longitude": longitude,
                             "Categories": categories}

            business_list.append(business_dict)

        except KeyError:

            pass

    business_df = pd.DataFrame(business_list)
    return business_df

df_2 = yelp_business_call(resp)

In [50]:
sw = {"lat": 25.616718, "lng": -100.484892}
ne = {"lat": 25.833780, "lng": -100.154103}
size = 50

lats = np.random.uniform(low = sw["lat"], high= ne["lat"], size=(size))
lngs = np.random.uniform(low = ne["lng"], high= sw["lng"], size=(size))
locs = list(zip(lats, lngs))

gmaps.configure(api_key=g_key)

marker_layer = gmaps.marker_layer(locs)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [90]:
#hotel yelp call

output_file = "mty_hotels.csv"

hotel_data = pd.DataFrame(columns = ['Name', 'Price', 'Rating', 'Review Count', 'City', 'Country',
       'Latitude', 'Longitude', 'Categories'])

log_counter = 1

for (latitude, longitude) in locs:
    
    params = {"latitude": latitude,
              "longitude": longitude,
              'term': 'hotel',
              "limit": 50,
              "radius": 10000}
    
    resp = requests.get(url=url, params=params, headers=headers).json()
    
    call_df = yelp_business_call(resp)
    data = pd.concat([data, call_df])
    
    print(f"Call No : {log_counter}")
    log_counter += 1
    
data = data.drop_duplicates(subset = ['Name', 'Price', 'Rating', 'Review Count', 'City', 'Country','Latitude', 'Longitude'])
data.to_csv(output_file, index = False)
data

Call No : 1
Call No : 2
Call No : 3
Call No : 4
Call No : 5
Call No : 6
Call No : 7
Call No : 8
Call No : 9
Call No : 10
Call No : 11
Call No : 12
Call No : 13
Call No : 14
Call No : 15
Call No : 16
Call No : 17
Call No : 18
Call No : 19
Call No : 20
Call No : 21
Call No : 22
Call No : 23
Call No : 24
Call No : 25
Call No : 26
Call No : 27
Call No : 28
Call No : 29
Call No : 30
Call No : 31
Call No : 32
Call No : 33
Call No : 34
Call No : 35
Call No : 36
Call No : 37
Call No : 38
Call No : 39
Call No : 40
Call No : 41
Call No : 42
Call No : 43
Call No : 44
Call No : 45
Call No : 46
Call No : 47
Call No : 48
Call No : 49
Call No : 50


,Name,Price,Rating,Review Count,City,Country,Latitude,Longitude,Categories
0,Holiday Inn,$$,3.5,2,San Nicolás de Los Garza,MX,25.741370,-100.302425,[hotels]
1,Best Western Plus Monterrey Aeropuerto,$$,3.5,5,Apodaca,MX,25.770325,-100.189124,[hotels]
2,Wyndham Garden Monterrey Norte,$$,2.0,3,San Nicolas De Los Garz,MX,25.744481,-100.302805,[hotels]
3,Gran Hotel Ancira,$$,4.5,7,Monterrey,MX,25.665925,-100.312410,[hotels]
4,LQ Hotel by La Quinta Monterrey Centro,$$,5.0,2,Monterrey,MX,25.684665,-100.317871,[hotels]
5,Hotel Safi Royal Luxury,$$$,4.0,8,Monterrey,MX,25.673260,-100.319730,[hotels]
6,Holiday Inn,$$$,4.0,4,Monterrey,MX,25.676269,-100.288664,[hotels]
7,Crowne Plaza Monterrey,$$$,3.5,13,Monterrey,MX,25.664699,-100.314111,[hotels]
8,Sheraton Ambassador Hotel,$$,3.0,9,Monterrey,MX,25.666318,-100.313788,[hotels]
9,Fiesta America Pabellon M,$$$,5.0,1,Monterrey,MX,25.665580,-100.317112,[hotels]


In [91]:
#restaurant yelp call

output_file = "mty_restaurants.csv"

hotel_data = pd.DataFrame(columns = ['Name', 'Price', 'Rating', 'Review Count', 'City', 'Country',
       'Latitude', 'Longitude', 'Categories'])

log_counter = 1

for (latitude, longitude) in locs:
    
    params = {"latitude": latitude,
              "longitude": longitude,
              'term': 'restaurant',
              "limit": 50,
              "radius": 10000}
    
    resp = requests.get(url=url, params=params, headers=headers).json()
    
    call_df = yelp_business_call(resp)
    data = pd.concat([data, call_df])
    
    print(f"Call No : {log_counter}")
    log_counter += 1
    
data = data.drop_duplicates(subset = ['Name', 'Price', 'Rating', 'Review Count', 'City', 'Country','Latitude', 'Longitude'])
data.to_csv(output_file, index = False)
data

Call No : 1
Call No : 2
Call No : 3
Call No : 4
Call No : 5
Call No : 6
Call No : 7
Call No : 8
Call No : 9
Call No : 10
Call No : 11
Call No : 12
Call No : 13
Call No : 14
Call No : 15
Call No : 16
Call No : 17
Call No : 18
Call No : 19
Call No : 20
Call No : 21
Call No : 22
Call No : 23
Call No : 24
Call No : 25
Call No : 26
Call No : 27
Call No : 28
Call No : 29
Call No : 30
Call No : 31
Call No : 32
Call No : 33
Call No : 34
Call No : 35
Call No : 36
Call No : 37
Call No : 38
Call No : 39
Call No : 40
Call No : 41
Call No : 42
Call No : 43
Call No : 44
Call No : 45
Call No : 46
Call No : 47
Call No : 48
Call No : 49
Call No : 50


,Name,Price,Rating,Review Count,City,Country,Latitude,Longitude,Categories
0,Holiday Inn,$$,3.5,2,San Nicolás de Los Garza,MX,25.741370,-100.302425,[hotels]
1,Best Western Plus Monterrey Aeropuerto,$$,3.5,5,Apodaca,MX,25.770325,-100.189124,[hotels]
2,Wyndham Garden Monterrey Norte,$$,2.0,3,San Nicolas De Los Garz,MX,25.744481,-100.302805,[hotels]
3,Gran Hotel Ancira,$$,4.5,7,Monterrey,MX,25.665925,-100.312410,[hotels]
4,LQ Hotel by La Quinta Monterrey Centro,$$,5.0,2,Monterrey,MX,25.684665,-100.317871,[hotels]
...,...,...,...,...,...,...,...,...,...
22,Los Bainas,$,5.0,1,Monterrey,MX,25.626900,-100.298859,[tacos]
23,Chorlitos,$$,5.0,1,Ampliación Valle del Mirador,MX,25.640957,-100.315796,[mexican]
25,Il Capriccio,$$$,5.0,12,San Pedro Garza García,MX,25.648409,-100.360249,[italian]
29,El Reten,$,4.0,1,Monterrey,MX,25.624787,-100.299490,[burgers]
